# ChatGLM大模型开发

开发调试说明：本地GPU不足，没有部署ChatGLM,采用下面两种方式进行开发调试
1.直接在云服务器中运行Jupyter进行开发测试。
2.本地Pycharm连接远程服务器,实现远程服务器本地调试。

## 1. ChatGLM3-6B模型API调用

步骤1： 导入相关的库

In [9]:
from transformers import AutoTokenizer, AutoModel

步骤2：加载tokenizer，首次加载需要的时间较长
使用 AutoTokenizer.from_pretrained 方法，加载预训练的tokenizer "THUDM/chatglm3-6b" 。
trust_remote_code=True 表示信任远程代码。

使用Hugging Face上开源的预训练模型，不依赖于chatglm3工程源代码

In [10]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b",
trust_remote_code=True)

步骤3：加载预训练模型
（这里是直接使用Hugging Face的模型库中开源的预训练模型进行开发测试）
使用 AutoModel.from_pretrained 方法，加载预训练的模型 "THUDM/chatglm3-6b" 到CUDA设
备上。 trust_remote_code=True 表示信任远程代码（如果有）， device='cuda' 表示将模型加载到CUDA设备上以便使用GPU加速

要注意的是，根据显卡显存的不同，需要考虑加载不同精度的模型。13GB显存以上的显卡可以直接按照上述代码加载全精度的模型。

In [ ]:
model = AutoModel.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True,device='cuda')

In [11]:
model = AutoModel.from_pretrained("THUDM/chatglm3-6b",trust_remote_code=True).quantize(8).cuda()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
model = AutoModel.from_pretrained("THUDM/chatglm3-6b",trust_remote_code=True).quantize(4).cuda()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

NameError: name 'round_up' is not defined

步骤4：实例化模型
接下来则需要对模型进行实例化操作，并且设置为评估模式：

In [12]:
model = model.eval()

步骤5：调用模型并获取结果

In [13]:
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

你好👋！我是人工智能助手 ChatGLM3-6B，很高兴见到你，欢迎问我任何问题。


In [14]:
print(history)

[{'role': 'user', 'content': '你好'}, {'role': 'assistant', 'metadata': '', 'content': '你好👋！我是人工智能助手 ChatGLM3-6B，很高兴见到你，欢迎问我任何问题。'}]


如果有多张 GPU，但是每张 GPU 的显存大小都不足以容纳完整的模型，那么可以将模型切分在多张GPU上。首先安装 accelerate: pip install accelerate，
然后通过如下方法加载模型：

In [ ]:
from utils import load_model_on_gpus
model = load_model_on_gpus("THUDM/chatglm3-6b", num_gpus=2)

## 2. OpenAI风格代码调用

In [ ]:
# pip install openai

OpenAI风格代码调用，首先启动 python openai_api.py

In [7]:
## OpenAI风格代码调用，首先启动 python openai_api.py

# 使用curl命令测试返回
# curl -X POST "http://127.0.0.1:8000/v1/chat/completions" \
# -H "Content-Type: application/json" \
# -d "{\"model\": \"chatglm3-6b\", \"messages\": [{\"role\": \"system\", \"content\": \"You are ChatGLM3, a large language model trained by Zhipu.AI. Follow the user's instructions carefully. Respond using markdown.\"}, {\"role\": \"user\", \"content\": \"你好，给我讲一个故事，大概100字\"}], \"stream\": false, \"max_tokens\": 100, \"temperature\": 0.8, \"top_p\": 0.8}"

# 使用Python代码测返回
import requests
import json

base_url = "http://127.0.0.1:8000" # 本地部署的地址,或者使用你访问模型的API地址

def create_chat_completion(model, messages):
    data = {
        "model": model, # 模型名称
        "messages": messages, # 会话历史
        "max_tokens": 100, # 最多生成字数
        "temperature": 0.8, # 温度
        "top_p": 0.8, # 采样概率
    }

    response = requests.post(f"{base_url}/v1/chat/completions", json=data)
    decoded_line = response.json()
    content = decoded_line.get("choices", [{}])[0].get("message", "").get("content", "")
    return content

代码调用

In [8]:
chat_messages = [
        {
            "role": "system",
            "content": "You are ChatGLM3, a large language model trained by Zhipu.AI. Follow the user's instructions carefully. Respond using markdown.",
        },
        {
            "role": "user",
            "content": "你好，给我讲一个故事，大概100字"
        }
    ]
content = create_chat_completion("chatglm3-6b", chat_messages)

In [9]:
print(content)


 从前有个叫小明的小男孩，他非常喜欢看动画片。有一天，他在电视上看到一则关于神奇世界的广告，于是激动地告诉了他的妈妈。妈妈带他去了一家图书馆， where他们借了许多关于魔法和奇幻的书籍。小明在阅读这些书籍时，发现了一个名为“小飞侠”的角色，他决定要成为一个勇敢的宇航员，探索未知的星球。从此以后，小明常常想象自己是一个勇敢的宇航员，在太空中


## 3. function calling功能

### 3.1 获取天气API演示

In [19]:
#open_weather_key = os.getenv("OPENWEATHER_API_KEY")
open_weather_key = "c6748580ef08742adf7bd05d79dd8e5d"

In [20]:
import json
import requests
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [21]:
data=get_weather("BeiJing")

In [22]:
data

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\\u6674\\uff0c\\u5c11\\u4e91", "icon": "02d"}], "base": "stations", "main": {"temp": 15.94, "feels_like": 14.09, "temp_min": 15.94, "temp_max": 15.94, "pressure": 1011, "humidity": 19, "sea_level": 1011, "grnd_level": 1005}, "visibility": 10000, "wind": {"speed": 4.98, "deg": 299, "gust": 8.94}, "clouds": {"all": 12}, "dt": 1700633608, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1700607993, "sunset": 1700643269}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

### 3.2 直接跟大模型问天气

In [17]:
##测试模型
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b",
trust_remote_code=True)

model = AutoModel.from_pretrained("THUDM/chatglm3-6b",trust_remote_code=True).quantize(8).cuda()

model = model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [23]:
## 测试模型调用
response, history = model.chat(tokenizer, "你好，北京天气怎么样？", history=[])
print(response)

你好！作为一个人工智能助手，我无法实时获取北京的最新天气信息。但你可以查看天气预报或使用天气应用来了解当前的北京天气。


### 3.3 一步一步演示function调用流程

In [24]:
## 定义function 函数
weather_api_spec = [
    {
        'name': 'get_weather',
        'description': '查询即时天气函数，根据输入的城市名称，查询对应城市的实时天气',
        'parameters': {
            'type': 'object',
            'properties': {
                'loc': {
                    'description': "城市名称，注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'",
                    'type': 'string',
                    'required': True
                }
            }
        }
        }
]

In [25]:
## 些system角色的prompt以及添加tools
system_info = {"role": "system", 
               "content": "Answer the following questions as best as you can. You have access to the following tools:", 
               "tools": weather_api_spec}


In [26]:
## 测试一下，添加了function以后模型调用的效果
history = [system_info]
query = "北京今天的天气"
function_call, history = model.chat(tokenizer, query, history=history)
## 识别出来去调用如下的函数
print(function_call)
print("==============")
print(history)

{'name': 'get_weather', 'parameters': {'loc': 'Beijing'}}
[{'role': 'system', 'content': 'Answer the following questions as best as you can. You have access to the following tools:', 'tools': [{'name': 'get_weather', 'description': '查询即时天气函数，根据输入的城市名称，查询对应城市的实时天气', 'parameters': {'type': 'object', 'properties': {'loc': {'description': "城市名称，注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'", 'type': 'string', 'required': True}}}}]}, {'role': 'user', 'content': '北京今天的天气'}, {'role': 'assistant', 'metadata': 'get_weather', 'content': " ```python\ntool_call(loc='Beijing')\n```"}]


In [27]:
## 获取函数信息，获取到被调用函数的函数名
function_name=function_call["name"]
function_name

'get_weather'

In [28]:
function_call["parameters"]["loc"]

'Beijing'

In [29]:
## 往函数里面添加函数的实现（真正逻辑的实现，去访问获取天气的代码）
functions_list = [get_weather]
available_functions = {func.__name__: func for func in functions_list}

In [30]:
fuction_to_call = available_functions[function_name]
fuction_to_call

<function __main__.get_weather(loc)>

In [31]:
# 获取函数参数
function_args = function_call['parameters']
function_args

{'loc': 'Beijing'}

In [32]:
function_response = fuction_to_call(**function_args)

In [33]:
function_response

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\\u6674\\uff0c\\u5c11\\u4e91", "icon": "02d"}], "base": "stations", "main": {"temp": 15.94, "feels_like": 14.09, "temp_min": 15.94, "temp_max": 15.94, "pressure": 1011, "humidity": 19, "sea_level": 1011, "grnd_level": 1005}, "visibility": 10000, "wind": {"speed": 4.98, "deg": 299, "gust": 8.94}, "clouds": {"all": 12}, "dt": 1700634535, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1700607993, "sunset": 1700643269}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

In [34]:
## 添加观察者的角色，name:是函数名称
## content：调用函数的信息
# user
# system
# assistant
# observation
history=[]
history.append(
    {
        "role": "observation",
        "name": function_name,
        "content": function_response,
    }
)  

In [35]:
## 模型调用
query = "请帮我查询一下北京的天气"
response, history = model.chat(tokenizer, query, history=history)

In [36]:
##打印返回的结果
print(response)

北京当前的天气情况如下：

- 温度：15.94℃，体感温度：14.09℃
- 天气状况：Clouds
- 湿度：19%
- 气压：1011 hPa
- 海平面高度：1011 m
- 风速：4.98 m/s
- 风向：299°

北京所在的地点是stations，当前时间是2022年9月27日17:08。


### 3.4 可以自己尝试封装为函数

In [37]:
def run_conv_glm(query,tokenizer, history, model,functions_list=None, functions=None, return_function_call=True):
    """
    能够自动执行外部函数调用的Chat对话模型
    :param messages: 必要参数，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为chatglm3-6b
    :return：Chat模型输出结果
    """

    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response, history = model.chat(tokenizer, query, history=history)
        final_response = response
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建调用外部函数的system_message
        system_info = {
            "role": "system",
            "content": "Answer the following questions as best as you can. You have access to the following tools:",
            "tools": functions,
        }
        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}
        history=[system_info]
        ## 第一次调用，目的是获取函数信息
        response,history = model.chat(tokenizer, query, history=history)
        # 需要调用外部函数
        function_call = response
        # 获取函数名
        function_name = function_call["name"]
        # 获取函数对象
        fuction_to_call = available_functions[function_name]
        # 获取函数参数
        function_args = function_call['parameters']
        # 将函数参数输入到函数中，获取函数计算结果
        function_response = fuction_to_call(**function_args)
        ## 第二次调用，带入进去函数
        # role="observation" 表示输入的是工具调用的返回值而不是用户输入
        # role:user,system,assistant,observation
        #
        #
        print(function_response)
        history=[]
        history.append(
                {
                    "role": "observation",
                    "name": function_name,
                    "content": function_response,
                }
        )  
        response, history = model.chat(tokenizer, query, history=history)
        final_response=response
    
    return final_response,history

In [ ]:
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b",
# trust_remote_code=True)

# model = AutoModel.from_pretrained("THUDM/chatglm3-6b",trust_remote_code=True).quantize(4).cuda()

# model = model.eval()

In [38]:
query = "请帮我查询一下北京的天气"
history=[]
functions_list = [get_weather]
functions=weather_api_spec

response,history = run_conv_glm(query=query,functions=functions,model=model,functions_list=functions_list,history=history,tokenizer=tokenizer)
print(response)

{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\u6674\uff0c\u5c11\u4e91", "icon": "02d"}], "base": "stations", "main": {"temp": 15.94, "feels_like": 14.09, "temp_min": 15.94, "temp_max": 15.94, "pressure": 1011, "humidity": 19, "sea_level": 1011, "grnd_level": 1005}, "visibility": 10000, "wind": {"speed": 4.98, "deg": 299, "gust": 8.94}, "clouds": {"all": 12}, "dt": 1700634535, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1700607993, "sunset": 1700643269}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}
北京现在的天气是：Clouds，温度为15.94℃，湿度为19%。


## 4. 智能SQL平台

### 4.1 数据库准备 

In [ ]:
CREATE TABLE user_info (
customerID VARCHAR(255),
gender VARCHAR(255),
SeniorCitizen INT,
Partner VARCHAR(255),
Dependents VARCHAR(255)
);


In [ ]:
INSERT INTO user_info (customerID, gender, SeniorCitizen, Partner, Dependents)
VALUES
('1', 'Female', 0, 'Yes', 'No'),
('2', 'Male', 1, 'No', 'Yes'),
('3', 'Male', 0, 'No', 'No'),
('4', 'Female', 1, 'Yes', 'Yes'),
('5', 'Male', 0, 'No', 'No'),
('6', 'Female', 0, 'Yes', 'Yes'),
('7', 'Male', 1, 'Yes', 'No'),
('8', 'Female', 0, 'No', 'No'),
('9', 'Male', 1, 'Yes', 'Yes'),
('10', 'Female', 0, 'No', 'No'),
('11', 'Male', 0, 'Yes', 'Yes'),
('12', 'Female', 1, 'No', 'No'),
('13', 'Male', 0, 'No', 'Yes'),
('14', 'Female', 0, 'Yes', 'No'),
('15', 'Male', 1, 'Yes', 'Yes'),
('16', 'Female', 0, 'No', 'No'),
('17', 'Male', 0, 'No', 'Yes'),
('18', 'Female', 1, 'Yes', 'No'),
('19', 'Male', 0, 'No', 'No'),
('20', 'Female', 1, 'No', 'Yes');

### 4.2 SQL执行代码封装

In [41]:
! pip install pymysql

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 44 kB 872 kB/s eta 0:00:01


In [42]:
import pymysql
import json
def sql_inter(sql_query):
    """
    用于执行一段SQL代码，并最终获取SQL代码执行结果，\
    核心功能是将输入的SQL代码传输至MySQL环境中进行运行，\
    并最终返回SQL代码运行结果。需要注意的是，本函数是借助pymysql来连接MySQL数据库。
    :param sql_query: 字符串形式的SQL查询语句，用于执行对MySQL中telco_db数据库中各张表进行查询，并获得各表中的各类相关信息
    :return：sql_query在MySQL中的运行结果。
    """
    
    
    connection = pymysql.connect(
            host="localhost",  # 数据库地址
            user='glm',  # 数据库用户名
            passwd="glm",  # 数据库密码
            db='chatglm_db',  # 数据库名
            charset='utf8'  # 字符集选择utf8
        )
    
    try:
        with connection.cursor() as cursor:
            # SQL查询语句
            sql = sql_query
            cursor.execute(sql)

            # 获取查询结果
            results = cursor.fetchall()

    finally:
        connection.close()
    
    
    return json.dumps(results)

In [43]:
sql_inter("select count(*) from user_info")

'[[20]]'

### 4.3 function call函数封装

In [48]:
sql_inter_function_info = [
    {
    'name': 'sql_inter',
    'description': '用于执行一段SQL代码，并最终获取SQL代码执行结果，核心功能是将输入的SQL代码传输至MySQL环境中进行运行，并最终返回SQL代码运行结果。',
    'parameters': {
        'type': 'object',
        'properties': {
            'sql_query': {
                'type': 'string',
                'description': '字符串形式的SQL代码，可以在MySQL中运行，并获取运行结果'
            }
        },
        'required': ['sql_query']
    }
}
]

### 4.4 读取本地知识库

In [44]:
# 打开并读取Markdown文件
with open('user_info.md', 'r', encoding='utf-8') as f:
    data_dictionary = f.read()

In [45]:
## 数据库测试
sql_inter(sql_query='SELECT COUNT(*) FROM user_info;')

'[[20]]'

### 4.5 函数封装 

In [46]:
def run_conv_glm(query,tokenizer, history, model,functions_list=None, functions=None, return_function_call=True):
    """
    能够自动执行外部函数调用的Chat对话模型
    :param messages: 必要参数，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为chatglm3-6b
    :return：Chat模型输出结果
    """

    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response, history = model.chat(tokenizer, query, history=history)
        final_response = response
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建调用外部函数的system_message
        system_info = {
            "role": "system",
            "content": "Answer the following questions as best as you can. You have access to the following tools:",
            "tools": functions,
        }
        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}
        history=[system_info]
        ## 第一次调用，目的是获取函数信息
       
        response,history = model.chat(tokenizer, query, history=history)
        print(response)
        # 需要调用外部函数
        function_call = response
        # 获取函数名
        function_name = function_call["name"]
        # 获取函数对象
        fuction_to_call = available_functions[function_name]
        # 获取函数参数
        function_args = function_call['parameters']
        # 将函数参数输入到函数中，获取函数计算结果
        function_response = fuction_to_call(**function_args)
        # print("答案")
        # print(function_response)
        # ## 第二次调用，带入进去函数
        # history=[]
        # history.append(
        #         {
        #             "role": "observation",
        #             "name": function_name,
        #             "content":function_response,
        #         }
        # ) 
        # print(history)
        # query= "请帮我到查询一下有多少电信用户，并给出答案"
        # response, history = model.chat(tokenizer, query, history=history)
        final_response=function_response
    
    return final_response,history

### 4.6 SQL调用

In [ ]:
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b",
# trust_remote_code=True)

# model = AutoModel.from_pretrained("THUDM/chatglm3-6b",trust_remote_code=True).quantize(4).cuda()

# model = model.eval()

In [54]:
query = data_dictionary + ",请帮我到查询一下有多少电信用户，并给出答案？"
history=[]
functions_list = [sql_inter]
functions=sql_inter_function_info
response,history = run_conv_glm(query=query,functions=functions,model=model,functions_list=functions_list,history=history,tokenizer=tokenizer)

{'name': 'sql_inter', 'parameters': {'sql_query': 'SELECT COUNT(*) FROM user_info'}}


In [55]:
print(response)

[[20]]
